In [3]:
%pip install requests

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 KB 1.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.3/167.3 KB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.3/126.3 KB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 KB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 KB 8.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
import requests
import json
import time
import os

In [9]:
secrets_file = "secrets.json"

if not os.path.exists(secrets_file):
    raise FileNotFoundError(f"{secrets_file} not found. Please create the file with your API key.")

with open(secrets_file, "r") as file:
    secrets = json.load(file)

API_KEY = secrets.get("google_api_key")

if not API_KEY:
    raise ValueError("API key not found in secrets.json. Please add your key under 'google_api_key'.")

In [ ]:
BASE_URL = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


In [11]:
# Bounding box for Calgary (approximate)
# Southwest corner: 50.8429, -114.3217
# Northeast corner: 51.1895, -113.8573
# ~50km, ~60km
SW_LAT, SW_LNG = 50.8429, -114.3217
NE_LAT, NE_LNG = 51.1895, -113.8573

# Grid division parameters
GRID_DIVISIONS = 10  # Number of divisions in each direction
RADIUS = 3000  # Search radius in meters for each grid cell

# Generate grid of center points
lat_step = (NE_LAT - SW_LAT) / GRID_DIVISIONS
lng_step = (NE_LNG - SW_LNG) / GRID_DIVISIONS
grid_centers = [
    (SW_LAT + i * lat_step, SW_LNG + j * lng_step)
    for i in range(GRID_DIVISIONS)
    for j in range(GRID_DIVISIONS)
]

In [29]:
def fetch_results(url, params):
    """
    Fetches data from the given URL with provided parameters.
    """
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response}")
        return None

In [32]:
# Collect all place IDs
all_place_ids = set()
all_places = []
queries = 0
for center in grid_centers:
    print(f"Querying center: {center}")
    params = {
        "location": f"{center[0]},{center[1]}",
        "radius": RADIUS,
        "type": "restaurant",
        "key": API_KEY
    }
    
    next_page_token = None
    while True:
        if next_page_token:
            params["pagetoken"] = next_page_token
            time.sleep(2)  # Delay as per Google API guidelines

        data = fetch_results(BASE_URL, params)
        queries += 1
        print("Query Number: ", queries)
        
        if data['status'] == 'OK':
            all_places.extend(data['results'])

            for place in data["results"]:
                all_place_ids.add(place["place_id"])

            output_file = "resturaunt_ids.json"
            with open(output_file, "w", encoding="utf-8") as f:
                json.dump(list(all_place_ids), f, indent=4)

            output_file = "resturaunts_details.json"
            with open(output_file, "w", encoding="utf-8") as f:
                json.dump(list(all_places), f, indent=4)

            if "next_page_token" in data:
                next_page_token = data.get("next_page_token")
            else: 
                break
        else: 
            break

Querying center: (50.8429, -114.3217)
Query Number:  1
Querying center: (50.8429, -114.27526)
Query Number:  2
Querying center: (50.8429, -114.22882)
Query Number:  3
Querying center: (50.8429, -114.18238000000001)
Query Number:  4
Querying center: (50.8429, -114.13594)
Query Number:  5
Querying center: (50.8429, -114.0895)
Query Number:  6
Querying center: (50.8429, -114.04306)
Query Number:  7
Query Number:  8
Querying center: (50.8429, -113.99662)
Query Number:  9
Querying center: (50.8429, -113.95018)
Query Number:  10
Querying center: (50.8429, -113.90374)
Query Number:  11
Querying center: (50.87756, -114.3217)
Query Number:  12
Querying center: (50.87756, -114.27526)
Query Number:  13
Querying center: (50.87756, -114.22882)
Query Number:  14
Querying center: (50.87756, -114.18238000000001)
Query Number:  15
Querying center: (50.87756, -114.13594)
Query Number:  16
Querying center: (50.87756, -114.0895)
Query Number:  17
Query Number:  18
Querying center: (50.87756, -114.04306)
Q

In [33]:
output_file = "resturaunt_ids.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(list(all_place_ids), f, indent=4)

output_file = "resturaunts_details.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(list(all_places), f, indent=4)